# BTM

This notebook implements Biterm Topic Model algorithm and tries to find the best parameters.

## Imports

In [4]:
import bitermplus as btm
import numpy as np
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
import gensim
from tqdm import tqdm

## Load data

In [5]:
def import_data():
    """Loads data and returns my_df (dataframe)
    Returns:
        my_df : the dataframe containing the data 
    """
    my_df = pd.read_csv('../data/to_be_clustered.csv.gz', compression="gzip")
    my_df = pd.DataFrame(my_df)
    return my_df

def clean_data(my_df, country):
    """Cleans the dataframe and keeps only rows and columns we are interested in
    Args:
        my_df : the dataframe to clean
        country : the country whose censored tweets we want to keep
    Returns:
        texts : the list of censored tweets in text form
    """
    my_df['clean'].apply(lambda x: str(x))
    in_df = my_df[my_df['whcs'] == country]
    in_df = in_df['clean'].dropna()
    texts = in_df.str.strip().tolist()
    
    return texts

## Preprocessing

In [6]:
def btm_preprocessing(texts):
    """Does some final preprocessing in order to use the data for BTM 
    Args:
        texts : the list of censored tweets in text form
    Returns:
        X : documents vs words matrix 
        vocabulary : Vocabulary (a list of words)
        biterms : list of biterms for each document
        docs_vec : vectorised documents (list of numpy.ndarray objects with terms ids)
    """
    # Obtaining terms frequency in a sparse matrix and corpus vocabulary
    X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
    tf = np.array(X.sum(axis=0)).ravel()
    # Vectorizing documents
    docs_vec = btm.get_vectorized_docs(texts, vocabulary)
    docs_lens = list(map(len, docs_vec))
    # Generating biterms
    biterms = btm.get_biterms(docs_vec)
    
    return X, vocabulary, biterms, docs_vec

## Create model

In [7]:
def create_model(X, vocabulary, seed, alpha, beta, biterms, docs_vec):
    """Creates the BTM model 
    Args:
        X : documents vs words frequency matrix
        vocabulary : vocabulary (a list of words)
        seed : random state seed
        alpha : model parameter
        beta : model parameter
        biterms : biterms list
        docs_vec : documents list. Each document must be presented as a list of words ids
    Returns:
        model : BTM model for the chosen parameters
    """
    model = btm.BTM(
    X, vocabulary, seed=seed, T=10, M=20, alpha=alpha, beta=beta)
    model.fit(biterms, iterations=20)
    p_zd = model.transform(docs_vec)
    
    return model

## Metrics

In [1]:
def compute_coherence(my_df, model):
    """Computes the coherence metric 
    Args:
        my_df : the original dataframe
        model : the BTM model
    Returns:
        coherence : the coherence value for this model
    """
    # cast tweets to numpy array
    docs = my_df.clean.apply(lambda x: str(x).split()).to_numpy()
    # create dictionary of all words in all documents
    dictionary = gensim.corpora.Dictionary(docs)
    # create BOW dictionary
    bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
    top_words = btm.get_top_topic_words(model, words_num=10)
    cm_gsdmm = CoherenceModel(topics=top_words.T.to_numpy(), 
                          dictionary=dictionary, 
                          corpus=bow_corpus,
                          texts=docs, 
                          coherence='c_v')
    coherence = cm_gsdmm.get_coherence()  # get coherence value
    
    return coherence

## Find parameters

In [37]:
def grid_search(my_df, low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta, X, vocabulary, biterms, docs_vec):
    """Computes a gridsearch to find the best parameters with respect to coherence metric
    Args:
       my_df : the original dataframe
       low_seed : the lowest value for seed
       high_seed : the highest value for seed
       low_alpha : the lowest value for alpha
       high_alpha : the highest value for alpha
       low_beta : the lowest value for beta
       high_beta : the highest value for beta
       X : documents vs words matrix
       vocabulary : vocabulary (a list of words)
       biterms : list of biterms for each document
       docs_vec : vectorised documents 
    Returns:
        best : the best obtained coherence value
        best_param : the corresonding seed, alpha and beta for best coherence value
    """
    best=0
    best_param = {}
    for seed in range(low_seed, high_seed, 1000):
        for alpha in np.logspace(low_alpha, high_alpha, 7):
            for beta in np.logspace(low_beta, high_beta, 5):
                model = create_model(X, vocabulary, seed, alpha, beta, biterms, docs_vec)
                coherence = compute_coherence(my_df, model)
                if (coherence > best) :
                    best = coherence
                    best_param['seed'] = seed
                    best_param['alpha'] = alpha
                    best_param['beta'] = beta
                    
    return best, best_param
        

In [38]:
def simulate(low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta):
    """Executes useful functions from loading data to having the best parameters
    Args:
       low_seed : the lowest value for seed
       high_seed : the highest value for seed
       low_alpha : the lowest value for alpha
       high_alpha : the highest value for alpha
       low_beta : the lowest value for beta
       high_beta : the highest value for beta 
    Returns:
        best : the best obtained coherence value
        best_param : the corresonding seed, alpha and beta for best coherence value
    """
    my_df = import_data()
    texts = clean_data(my_df, "France")
    X, vocabulary, biterms, docs_vec = btm_preprocessing(texts)
    best, best_param = grid_search(my_df, low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta, X, vocabulary, biterms, docs_vec)
    print('best', best)
    print('best params', best_param)
    return best, best_param

In [39]:
best, best_param = simulate(1, 12000, -6, 2, -6, 0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 48867.58it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 92043.15it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 93965.44it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 90973.96it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 93516.17it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 93034.47it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 73908.44it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 59136.71it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 79837.43it/s]


best 0.5040345516844125
best params {'seed': 6001, 'alpha': 4.641588833612772, 'beta': 1.0}


In [40]:
best
#0.5040345516844125

0.5040345516844125

In [41]:
best_param
#{'seed': 6001, 'alpha': 4.641588833612772, 'beta': 1.0}

{'seed': 6001, 'alpha': 4.641588833612772, 'beta': 1.0}

## Reproduce with best parameters

In [47]:
def best_model():
    """Reproduces the model with best parameters"""
    my_df = import_data()
    texts = clean_data(my_df, "France")
    X, vocabulary, biterms, docs_vec = btm_preprocessing(texts)
    model = create_model(X, vocabulary, 6001, 4.641588833612772, 1, biterms, docs_vec)
    coherence = compute_coherence(my_df, model)
    print(coherence)
    return model

In [48]:
best_m = best_model()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 82651.81it/s]


-7.377470466087462
